# CBLOL Player Stats

#### Importing libraries

In [79]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd

#### Path variables

In [80]:
URL = """
https://lol.fandom.com/wiki/Special:RunQuery\
/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer&\
TS%5Btournament%5D={}&\
TS%5Blink%5D=&TS%5Bchampion%5D=&TS%5Brole%5D=&TS%5Bteam%5D=&TS%5Bpatch%5D=&TS%5Byear%5D=&TS%5Bregion%5D=&TS%5Btournamentlevel%5D=&TS%5Bwhere%5D=&TS%5Bincludelink%5D%5Bis_checkbox%5D=true&TS%5Bshownet%5D%5Bis_checkbox%5D=true&_run=&pfRunQueryFormName=TournamentStatistics&wpRunQuery=&pf_free_text=\
"""
TOURNAMENTS = [
    "CBLOL%2F2014+Season%2FChampions+Series", 
    "CBLOL%2F2014+Season%2FRegional+Finals",
    "CBLOL%2F2015+Season%2FSplit+1", 
    "CBLOL%2F2015+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2015+Season%2FSplit+2", 
    "CBLOL%2F2015+Season%2FSplit+2+Playoffs", 
    "CBLOL%2F2016+Season%2FSplit+1", 
    "CBLOL%2F2016+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2016+Season%2FSplit+2", 
    "CBLOL%2F2016+Season%2FSplit+2+Playoffs", 
    "CBLOL%2F2017+Season%2FSplit+1", 
    "CBLOL%2F2017+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2017+Season%2FSplit+2", 
    "CBLOL%2F2017+Season%2FSplit+2+Playoffs",
    "CBLOL%2F2018+Season%2FSplit+1", 
    "CBLOL%2F2018+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2018+Season%2FSplit+2", 
    "CBLOL%2F2018+Season%2FSplit+2+Playoffs", 
    "CBLOL%2F2019+Season%2FSplit+1", 
    "CBLOL%2F2019+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2019+Season%2FSplit+2", 
    "CBLOL%2F2019+Season%2FSplit+2+Playoffs", 
    "CBLOL%2F2020+Season%2FSplit+1", 
    "CBLOL%2F2020+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2020+Season%2FSplit+2", 
    "CBLOL%2F2020+Season%2FSplit+2+Playoffs",
    "CBLOL%2F2021+Season%2FSplit+1", 
    "CBLOL%2F2021+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2021+Season%2FSplit+2", 
    "CBLOL%2F2021+Season%2FSplit+2+Playoffs",
    "CBLOL%2F2022+Season%2FSplit+1", 
    "CBLOL%2F2022+Season%2FSplit+1+Playoffs",
    "CBLOL%2F2022+Season%2FSplit+2", 
    "CBLOL%2F2022+Season%2FSplit+2+Playoffs",
    "CBLOL%2F2023+Season%2FSplit+1", 
]

#### Generating player stats

In [104]:
def get_page_table(PATH, TOURNAMENT):
    page_request = re.get(PATH.format(TOURNAMENT))
    soup = BeautifulSoup(page_request.content, 'html.parser')
    table = soup.find('table')
    return table

def fill_player_data(NAME, MP, WINS):
    player_name = NAME.lower()
    if player_name not in PLAYERS_STATS:
        PLAYERS_STATS[player_name] = [
            int(MP), 
            int(WINS)
        ]
    else:
        PLAYERS_STATS[player_name] = [
            PLAYERS_STATS[player_name][0] + int(MP), 
            PLAYERS_STATS[player_name][1] + int(WINS)
        ]

def get_cell_data_fill_player_data(ROW, ITH_ROW):
    cell_data = []
    for cell in ROW.find_all("td"):
        cell_data.append(cell.text)
    if ITH_ROW > 5:
        fill_player_data(cell_data[1], cell_data[2], cell_data[3])
    
def main():
    for t in range(len(TOURNAMENTS)):
        i = 0
        TABLE = get_page_table(URL, TOURNAMENTS[t])
        for result in TABLE:
            for row in result.find_all("tr"):
                i += 1
                get_cell_data_fill_player_data(row, i)


PLAYERS_STATS = {}
main()

#### Generating players dataframe 

In [109]:
df = pd.DataFrame(PLAYERS_STATS)

players_df = df.transpose()
players_df.columns = ["MP", "W"]
players_df["WR"] = round(players_df.W/players_df.MP, 4) * 100



#### Greater WR with at least N matches

In [111]:
def player_winrate_df(MIN_MP, QTY_PLAYERS):
    return players_df[players_df["MP"] >= MIN_MP].sort_values(by="WR", ascending=False)[["MP", "WR"]].head(QTY_PLAYERS)

player_winrate_df(50, 5)

,MP,WR
shrimp,85,75.29
luci,138,72.46
brtt,300,67.00
dioud,90,65.56
tockers,196,65.31


#### Greater MP

In [113]:
def player_matchesplayed_df(QTY_PLAYERS):
    return players_df.sort_values(by="MP", ascending=False)[["MP", "WR"]].head(QTY_PLAYERS)

player_matchesplayed_df(5)

,MP,WR
robo,371,63.88
micao,345,54.20
tinowns,323,54.18
brtt,300,67.00
redbert,287,54.01
